In [12]:
import pyvisa as pyv
from typing import List

class WaveformGenerator:
    def __init__(self, resource_string: str):
        self.rm = pyv.ResourceManager()
        self.device = self.rm.open_resource(resource_string)
        response = self.device.query('*IDN?')
        print("Instrument ID:", response)

    def set_waveform_type(self, channel: int, waveform_type: str) -> None:
        command = f'C{channel}:BSWV WVTP,{waveform_type}'
        self.device.write(command)

    def set_frequency(self, channel: int, frequency: float) -> None:
        command = f'C{channel}:BSWV FRQ,{frequency}'
        self.device.write(command)

    def set_amplitude(self, channel: int, amplitude: float) -> None:
        command = f'C{channel}:BSWV AMP,{amplitude}'
        self.device.write(command)

    def set_phase(self, channel: int, phase: float) -> None:
        command = f'C{channel}:BSWV PHSE,{phase}'
        self.device.write(command)

    def start_waveform(self, channel: int) -> None:
        self.device.write(f'C{channel}:OUTP ON')

    def stop_waveform(self, channel: int) -> None:
        self.device.write(f'C{channel}:OUTP OFF')

    def load_waveform_file(self, channel: int, file_path: str) -> None:
        self.device.write(f'C{channel}:BSWV WVTP,ARB')
        self.device.write(f'C{channel}:BSWV ARWV,"{file_path}"')
        self.start_waveform(channel)

def waveform_config(ip_string: str = None):
    if ip_string is None:
        type_of_connection = input("Enter connection type (i.e., IP, USB, etc.): ").upper()
        
        if type_of_connection == "IP":
            type_of_connection = "TCPIP"
        elif type_of_connection == "USB":
            type_of_connection = "USB"
        else:
            print("Invalid connection type. Please enter 'IP' or 'USB'.")
            return

        ip_address = input("Enter IP address: ")
        ip_string = f"{type_of_connection}::{ip_address}::INSTR"

    generator = WaveformGenerator(ip_string)

    while True:
        try:
            channel = int(input("Enter the channel number (e.g., 1 or 2): "))
            if channel not in [1, 2]:
                print("Invalid channel number. Please enter 1 or 2.")
                continue

            waveform_type = input("Enter a waveform type (i.e sine, square, triangle): ")
            generator.set_waveform_type(channel, waveform_type)

            frequency = float(input("Set frequency in Hz: "))
            generator.set_frequency(channel, frequency)

            amplitude = float(input("Set amplitude: "))
            generator.set_amplitude(channel, amplitude)

            print("Starting waveform")
            generator.start_waveform(channel)

            load_file = input("Do you want to load a custom waveform file from the U-disk0/wave directory? (yes/no): ").lower()
            if load_file == "yes":
                file_name = input("Enter the filename of the waveform file (e.g., custom_waveform.csv): ")
                file_path = f'U-disk0/wave/{file_name}'
                generator.load_waveform_file(channel, file_path)
                print(f"Custom waveform file {file_path} loaded and outputted successfully.")

            stop = input("Do you want to stop the waveform output? (yes/no, this exits both): ").lower()
            if stop == "yes":
                generator.stop_waveform(1)
                generator.stop_waveform(2)
                break

            restart = input("Do you want to reconfigure, or use another channel? (yes/no): ").lower()
            if restart != "yes":
                break

        except ValueError as e:
            print(f"Invalid input or error: {e}")

if __name__ == "__main__":
    waveform_config()


Instrument ID: Siglent Technologies,SDG6052X,SDG6XFCQ6R0831,6.01.01.36R3

Starting waveform
Custom waveform file U-disk0/wave/random_waveform_1.bin loaded and outputted successfully.
